In [7]:
from bs4 import BeautifulSoup
import requests
import re 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [78]:
driver = webdriver.Chrome()

In [3]:
base_url = "https://www.karaca.com/yatak-odasi?page="

In [25]:
for i in range(1,30):
    url = f"{base_url}{i}"
    driver.get(url)
    time.sleep(1)
    products = driver.find_elements(by=By.CSS_SELECTOR, value= "[class = 'productWrap']")  
    time.sleep(3)
    driver.execute_script('window.scrollBy(0, 2000)')   

In [79]:
img_list = []
title_list = []
href_list = []
price_list = []
id_list = []

for i in range(1,30):
    url = f"{base_url}{i}"
    driver.get(url)
    time.sleep(1)
    products = driver.find_elements(by=By.CSS_SELECTOR, value= "[class = 'productWrap']")  
    time.sleep(3)
    driver.execute_script('window.scrollBy(0, 2000)')   
    for product in products:
        soup = BeautifulSoup(product.get_attribute("outerHTML"), 'html.parser')
        image_tag = soup.find("img")
        image_src = image_tag['src']
        img_list.append(image_src)
        
        element = soup.findAll("a")
        
        
        title_src = element[0]['data-productname']
        title_list.append(title_src)
        
        id_src = element[0]['data-product-id']
        id_list.append(id_src)
        
        price_src = element[1]['data-productprice']
        price_list.append(price_src)
        
        href_src = element[1]['href']
        href_list.append(href_src)

In [80]:
ptt_data = {
    "id" : id_list,
    "title": title_list,
    "price": price_list,
    "link" : href_list,
    "img_url": img_list   
}
test_data = pd.DataFrame(ptt_data,index=None)

In [82]:
test_data.to_csv("karaca.csv",index=None)

In [83]:
test_data

,id,title,price,link,img_url
0,6218885,Karaca Home Melosa Pembe %100 Pamuk Çift Kişil...,1299,https://www.karaca.com/urun/karaca-home-melosa...,https://cdn.karaca.com/rcman/cw250h250q90gm/im...
1,7131147,Karaca Home Joena Bej Çift Kişilik Battaniye,424.99,https://www.karaca.com/urun/karaca-home-joena-...,https://cdn.karaca.com/rcman/cw250h250q90gm/im...
2,7175189,Karaca Home Raya Çift Kişilik Pamuk Battaniye ...,399.99,https://www.karaca.com/urun/karaca-home-raya-c...,https://cdn.karaca.com/rcman/cw250h250q90gm/im...
3,7166059,Karaca Home Tuğba 8 Parça Private Çeyiz Seti Gold,2999,https://www.karaca.com/urun/karaca-home-tugba-...,https://cdn.karaca.com/rcman/cw250h250q90gm/im...
4,4005791,Karaca Home Neos Mavi %100 Pamuk Çift Kişilik ...,799.99,https://www.karaca.com/urun/karaca-home-neos-m...,https://cdn.karaca.com/rcman/cw250h250q90gm/im...
...,...,...,...,...,...
691,7172766,Nautica Home Darla Tek Kişilik Pike Lacivert,399.99,https://www.karaca.com/urun/nautica-home-darla...,https://static.karaca.com/catalog/view/assets/...
692,7172765,Nautica Home Darla Çift Kişilik Pike Lacivert,449.99,https://www.karaca.com/urun/nautica-darla-cift...,https://static.karaca.com/catalog/view/assets/...
693,7172553,Nautica Home Marine Life %100 Pamuk Çift Kişil...,899.99,https://www.karaca.com/urun/nautica-home-marin...,https://static.karaca.com/catalog/view/assets/...
694,7143684,Karaca Home Wilma Lacivert Throw Battaniye 130...,249.99,https://www.karaca.com/urun/karaca-home-wilma-...,https://static.karaca.com/catalog/view/assets/...


In [61]:
price_list = []
for product in products:
    soup = BeautifulSoup(product.get_attribute("outerHTML"), 'html.parser')
    price = soup.findAll("a")
    price_src = price[1]['data-productprice']
    price_list.append(price_src)

In [87]:
cat_list = ["https://www.pttavm.com/arama/dizustu-bilgisayar?page="
            ]

In [88]:
ptt_url = "https://www.pttavm.com"
pagination_check = "0 ürün gösteriliyor"
title_list = []
link_list = []
ptt_cat_list = []
price_list= []
id_list = []

for base_url in cat_list:
    for i in range(1,50):
        url = f"{base_url}{i}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text,"html.parser")
        page_info = soup.find("span",class_= "md:text-base hidden md:block")
        b_tag = page_info.find('b')
        print(i)
        if b_tag.text == pagination_check:
            print("sayfa bitti")
            break
        
                    
       
        products = soup.find_all('div', class_='product-list-box-container-info')
        for product in products:
            prices = product.find_all('div', class_='price-box flex flex-1 flex-col-reverse price-box-min-h')
            link_tag = product.find('a')
            if link_tag:
                title_value = link_tag.get('title')
                link_value = link_tag.get("href")
            for price_tag in prices:
                price_text = price_tag.get_text(strip=True)
                id_filter = r'\d+-p'
                match = re.search(id_filter, link_value[::-1])
                
                price_list.append(price_text)
                link_list.append(f"{ptt_url}{link_value}")
                title_list.append(title_value)
                id_list.append(match.group()[::-1].replace("p-",""))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [110]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()


In [132]:
title_list = []
link_list = []
ptt_cat_list = []
price_list= []
id_list = []
img_list = []

for i in range(1,50):
    driver.get(f"https://www.pttavm.com/arama/dizustu-bilgisayar?page={i}")
    products_info = driver.find_elements(by=By.CSS_SELECTOR, value= "[class = 'product-list-box-container-info']")
    products = driver.find_elements(by=By.CSS_SELECTOR, value= "[class = 'product-list-box-container-slider']")


    for product in products:
        soup = BeautifulSoup(product.get_attribute("outerHTML"), 'html.parser')
        image_tag = soup.find("img")
        image_src = image_tag['src']
        img_list.append(image_src)
    
    for product in products_info:
        soup = BeautifulSoup(product.get_attribute("outerHTML"), 'html.parser')

        prices = soup.find_all('div', class_='price-box flex flex-1 flex-col-reverse price-box-min-h')
        link_tag = soup.find('a')
        if link_tag:
            title_value = link_tag.get('title')
            link_value = link_tag.get("href")
        for price_tag in prices:
            price_text = price_tag.get_text(strip=True)
            id_filter = r'\d+-p'
            match = re.search(id_filter, link_value[::-1])
            
            price_list.append(price_text)
            link_list.append(f"{ptt_url}{link_value}")
            title_list.append(title_value)
            id_list.append(match.group()[::-1].replace("p-",""))
        

In [109]:
# from selenium import webdriver
# from bs4 import BeautifulSoup
# from selenium.webdriver.common.by import By
# import time

# img_list = []

# driver = webdriver.Chrome()
# for i in range(1,2):
#     driver.get(f"https://www.pttavm.com/arama/dizustu-bilgisayar?page={i}")
#     driver.implicitly_wait(10)
#     products = driver.find_elements(by=By.CSS_SELECTOR, value= "[class = 'product-list-box-container-slider']")
#     products_info = driver.find_elements(by=By.CSS_SELECTOR, value= "[class = 'product-list-box-container-info']")

#     for product in products:
#         soup = BeautifulSoup(product.get_attribute("outerHTML"), 'html.parser')
#         image_tag = soup.find("img")
#         image_src = image_tag['src']
#         img_list.append(image_src)
        
        

        
#         for product in products_info:
#             soup = BeautifulSoup(product.get_attribute("outerHTML"), 'html.parser')
#             productss = soup.find_all('div', class_='product-list-box-container-info')
            
            
#             prices = productss.find_all('div', class_='price-box flex flex-1 flex-col-reverse price-box-min-h')
#             link_tag = productss.find('a')
#             if link_tag:
#                 title_value = link_tag.get('title')
#                 link_value = link_tag.get("href")
#             for price_tag in prices:
#                 price_text = price_tag.get_text(strip=True)
#                 id_filter = r'\d+-p'
#                 match = re.search(id_filter, link_value[::-1])
                
#                 price_list.append(price_text)
#                 link_list.append(f"{ptt_url}{link_value}")
#                 title_list.append(title_value)
#                 id_list.append(match.group()[::-1].replace("p-",""))



AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [133]:
ptt_data = {
    "id" : id_list,
    "title": title_list,
    "price": price_list,
    "link" : link_list,
    "img_url": img_list   
}
test_data = pd.DataFrame(ptt_data,index=None)

In [134]:
test_data

,id,title,price,link,img_url
0,734639731,LENOVO V15 82YU00QYTX RYZEN5-7520U 16GB 512SSD...,"14.511,80 TL",https://www.pttavm.com/lenovo-v15-82yu00qytx-r...,https://img-pttavm.mncdn.com/mnresize/244/244/...
1,568728058,"Lenovo V15 82YU00METX 7120U 8GB 256SSD 15.6"" F...","10.370,00 TL",https://www.pttavm.com/lenovo-v15-82yu00metx-7...,https://img-pttavm.mncdn.com/mnresize/244/244/...
2,710061769,Asus X415MA-EK385W04 N4020 8gb 256SSD 14 Fullh...,"9.699,03 TL",https://www.pttavm.com/asus-x415ma-ek385w04-n4...,https://img-pttavm.mncdn.com/mnresize/244/244/...
3,653067835,Lenovo Legion 5 Pro İntel Core i7 11800H 16GB ...,"45.599,04 TL",https://www.pttavm.com/lenovo-legion-5-pro-int...,https://img-pttavm.mncdn.com/mnresize/244/244/...
4,540848982,Asus X515EA-BQ868W i3-1115G4 4 GB 256 GB SSD U...,"9.599,00 TL",https://www.pttavm.com/asus-x515ea-bq868-i3-11...,https://img-pttavm.mncdn.com/mnresize/244/244/...
...,...,...,...,...,...
2347,729109370,Lenovo IdeaPad Gaming 3 Ryzen 5 5600H 32 GB 1 ...,"34.372,78 TL",https://www.pttavm.com/lenovo-ideapad-gaming-3...,https://img-pttavm.mncdn.com/mnresize/244/244/...
2348,729117560,DELL VOSTRO 3420 İntel Core i5 1235U 64gb 512g...,"22.079,99 TL",https://www.pttavm.com/dell-vostro-3420-intel-...,https://img-pttavm.mncdn.com/mnresize/244/244/...
2349,729118190,HP 250 G9 İNTEL CORE İ5 1235U 64gb 512gb SSD I...,"23.039,99 TL",https://www.pttavm.com/hp-250-g9-intel-core-i5...,https://img-pttavm.mncdn.com/mnresize/244/244/...
2350,729120710,HP 245 G9 AMD RYZEN 5 5625U 32gb 512gb SSD AMD...,"18.815,98 TL",https://www.pttavm.com/hp-245-g9-amd-ryzen-5-5...,https://img-pttavm.mncdn.com/mnresize/244/244/...


In [135]:
test_data.to_csv("data.csv",index=None)
test_data.to_excel("data.xlsx",index=None)